<a href="https://colab.research.google.com/github/nitin649/BERT_NER/blob/main/Bert_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn 
import os 
import pandas as pd
from sklearn import model_selection
from transformers import BertTokenizer
from transformers import BertModel
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
def fun_create_data():
  with open(r'/content/drive/My Drive/Colab Notebooks/NamesDataset/train.txt','r') as file:
    x=file.readlines()
   
    new_line_index=[]
    final_sents=[]
    original_tags=[]
    encoded_tags=[]
    tags_dict={}
    start=0
    count=-1
    tag_index=-1
    for i in x:
        count+=1
        if len(i)==1:
            new_line_index.append(count)
    for end in new_line_index[:5000]:
        sent=' '.join([word.split(' ')[0] for word in x[start:end]])
        tag=','.join([word.split(' ')[-1].replace('\n',' ') for word in x[start:end]])
        final_sents.append(sent.split(' '))
        original_tags.append(tag.split(','))
        for single_tag in tag.split(','):
          if single_tag not in tags_dict:
            tag_index+=1
            tags_dict[single_tag]=tag_index

        start=end+1
    print(tags_dict)
    for i in original_tags:
       new_tags=[]
       for j in i:
         new_tags.append(tags_dict[j])
       encoded_tags.append(new_tags)

    

    print(len(final_sents),len(original_tags),len(encoded_tags),tags_dict)
    return final_sents,original_tags,encoded_tags,len(tags_dict),tags_dict

In [ ]:
final_sents,original_tags,encoded_tags,num_classes,tags_dict=fun_create_data()

{'B-ORG ': 0, 'O ': 1, 'B-MISC ': 2, 'B-PER ': 3, 'I-PER ': 4, 'B-LOC ': 5, 'I-ORG ': 6, 'I-MISC ': 7, 'I-LOC ': 8}
5000 5000 5000 {'B-ORG ': 0, 'O ': 1, 'B-MISC ': 2, 'B-PER ': 3, 'I-PER ': 4, 'B-LOC ': 5, 'I-ORG ': 6, 'I-MISC ': 7, 'I-LOC ': 8}


In [ ]:
print(tags_dict)

{'B-ORG ': 0, 'O ': 1, 'B-MISC ': 2, 'B-PER ': 3, 'I-PER ': 4, 'B-LOC ': 5, 'I-ORG ': 6, 'I-MISC ': 7, 'I-LOC ': 8}


In [ ]:
print(len(final_sents))
print(len(original_tags))
print(len(encoded_tags))

5000
5000
5000


In [ ]:
final_sents[2450]

['American',
 'Dennis',
 'Mitchell',
 'upstaged',
 'a',
 'trio',
 'of',
 'past',
 'and',
 'present',
 'Olympic',
 '100',
 'metres',
 'champions',
 'on',
 'Friday',
 'with',
 'a',
 'storming',
 'victory',
 'at',
 'the',
 'Brussels',
 'grand',
 'prix',
 '.']

In [ ]:
original_tags[2450]

['B-MISC ',
 'B-PER ',
 'I-PER ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'B-MISC ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'O ',
 'B-LOC ',
 'O ',
 'O ',
 'O ']

In [ ]:
encoded_tags[2450]

[2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1]

In [ ]:
tags_dict

{'B-LOC ': 5,
 'B-MISC ': 2,
 'B-ORG ': 0,
 'B-PER ': 3,
 'I-LOC ': 8,
 'I-MISC ': 7,
 'I-ORG ': 6,
 'I-PER ': 4,
 'O ': 1}

In [ ]:

def max_sent_len(final_sents):
  max_sent=[]
  sent=[]
  for i in final_sents:
    for j in i:
      tokenizer=BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)
      x=tokenizer.tokenize(j)
      sent.extend(x)
    max_sent.append((len(sent)))
  return max(max_sent)
      

In [ ]:
#max_sent_len(final_sents=final_sents)

In [ ]:
from sklearn import model_selection
train_sents,test_sents,train_tags,test_tags=model_selection.train_test_split(final_sents,encoded_tags,random_state=42,test_size=0.1)

In [ ]:
print(len(train_sents))
print(len(test_sents))
print(len(train_tags))
print(len(test_tags))

4500
500
4500
500


In [ ]:
len(tags_dict)


9

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class NerDataset(Dataset):
    def __init__(self,sents,tags,max_len):
      #print(sents)
      #print(tags)
      self.sents=sents
      self.tags=tags
      self.max_len=max_len
      #self.device=device
      
    def __len__(self):
      return (len(self.sents))

    def __getitem__(self,index):
      self.tokenizer=BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)

      sent=self.sents[index]
      tag=self.tags[index]

      final_sents=[]
      final_tags=[]
      
      for i , s in enumerate(sent):
          tokenize_words=self.tokenizer.encode(
              s,
              add_special_tokens=False
          )
          input_len=len(tokenize_words)
          final_sents.extend(tokenize_words)
          final_tags.extend([tag[i]]*input_len)
      
      final_sents=final_sents[:self.max_len-2] #truncating
      final_tags=final_tags[:self.max_len-2]

      final_sents=[101] + final_sents + [102]
      final_tags=[0] + final_tags+[0]

      mask=[1] * len(final_sents)

      segment_ids=[0]*len(final_sents)
      

      #padding our data

      padding_len=self.max_len - len(final_sents)
      #print(padding_len)
      final_sents=final_sents +([0] * padding_len)
      final_tags=final_tags +([0] * padding_len)
      mask=mask+([0] * padding_len)
      segment_ids=segment_ids + ([0] * padding_len)

      #print(final_sents)
      #print(len(final_tags))
      #print(len(mask))
      #print(len(segment_ids))
      
      return {
          'id':torch.tensor(final_sents,dtype=torch.long),
          'target_tag':torch.tensor(final_tags,dtype=torch.long),
          'mask':torch.tensor(mask,dtype=torch.long),
          'token_type_ids':torch.tensor(segment_ids,dtype=torch.long)
      }


In [ ]:
max_sent_len=128
train_dataset=NerDataset(sents=train_sents,tags=train_tags,max_len=max_sent_len)
test_dataset=NerDataset(sents=test_sents,tags=test_tags,max_len=max_sent_len)

In [ ]:
128*5

640

In [ ]:
def save_checkpoint(state,filename="/content/drive/My Drive/Colab Notebooks/TransformerMachineTranslation/bert_ner_checkpoint/bert_checkpoint_second.pth"):
  print('saving checkpoint')
  torch.save(state,filename)


In [ ]:
def load_checkpoint(checkpoint):
  print('loading checkpoint')
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
load_model=False #whenever we have to load set this flag to true
if load_model:
  load_checkpoint(torch.load('/content/drive/My Drive/Colab Notebooks/TransformerMachineTranslation/bert_ner_checkpoint/bert_checkpoint_second.pth'))

In [ ]:
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1 #this  is used just for calculate loss just considering the original data not the padded one
    #output shapes [batch,entries,classes]
    active_logits = output.view(-1, num_labels) #converting into [entries , classes]
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    #print(active_logits.size(),active_labels.size())
    loss = lfn(active_logits, active_labels)
    return loss

class Model(nn.Module):
    def __init__(self,num_tags,hidden_size,input_size,num_layer):
      super().__init__()
      self.num_tags=num_tags
      self.bert = BertModel.from_pretrained('bert-base-cased')
      self.lstm=nn.LSTM(input_size,hidden_size=hidden_size,num_layers=num_layer,batch_first=True,bidirectional=True)
      self.hidden_size=hidden_size
      self.num_layer=num_layer
      self.bert_drop_1 = nn.Dropout(0.3)
      self.out_tag = nn.Linear(hidden_size * 2 , self.num_tags)
    
    def forward(self,id,mask,token_type_ids,target_tag):
      o1, _ = self.bert(id, attention_mask=mask, token_type_ids=token_type_ids)
      o1 = self.bert_drop_1(o1)
      h0=torch.zeros(self.num_layer*2,o1.size(0),self.hidden_size).to(device) #hidden state
      c0=torch.zeros(self.num_layer*2,o1.size(0),self.hidden_size).to(device) #hidden state
      o1,_=self.lstm(o1,(h0,c0))
      #o1=torch.reshape(o1,[-1,self.hidden_size*2])
      
      tag = self.out_tag(o1)
      loss_tag = loss_fn(tag, target_tag, mask, self.num_tags)

      #print('output tags',tag)
      return tag,loss_tag

In [ ]:
class Model_Normal(nn.Module):
    def __init__(self,num_tags):
      super().__init__()
      self.num_tags=num_tags
      self.bert = BertModel.from_pretrained('bert-base-cased')
      self.hidden_size=hidden_size
      self.bert_drop_1 = nn.Dropout(0.3)
      
      self.out_tag = nn.Linear(768 , self.num_tags)
    
    def forward(self,id,mask,token_type_ids,target_tag):
      o1, _ = self.bert(id, attention_mask=mask, token_type_ids=token_type_ids)
      o1 = self.bert_drop_1(o1)

      tag = self.out_tag(o1)
      loss_tag = loss_fn(tag, target_tag, mask, self.num_tags)

      #print('output tags',tag)
      return tag,loss_tag

In [ ]:
#hyperparameter
num_class =9
hidden_size=256
input_size=768
batch_size=32
num_layer=1
num_epocs=3
learning_rate=3e-5

In [ ]:
#model=Model(hidden_size=hidden_size,input_size=input_size,num_layer=num_layer,num_tags=num_class).to(device)

In [ ]:
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size)

In [ ]:
model=Model_Normal(num_tags=num_class).to(device)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
#criterion2=nn.CrossEntropyLoss()
#optimizer2=optim.Adam(model2.parameters(),lr=learning_rate)

In [ ]:
#for batch_idx , data in enumerate(train_loader):
  #print(batch_idx , data)

In [ ]:
#for epoch in range(10):
  #losses=[] #use for calculating loss at each epoch
  #model.train()

  #print(f'epoch number {epoch}')
  #checkpoint={'state_dict':model.state_dict(),'optimizer':optimizer.state_dict()}
  #save_checkpoint(checkpoint)
  #for batch_idx , data in enumerate(train_loader):
    #for k, v in data.items():
        #data[k] = v.to(device)
    #target_tag=data['target_tag']  
    #forward
    #_,loss=model(**data)
    #print(scores.size())
    #target_tag=target_tag.view(-1)
    #loss=criterion(scores,target_tag)

    #losses.append(loss.item())

    #backward
    #optimizer.zero_grad()
    #loss.backward()

    #optimizer.step()

  #mean_loss=sum(losses)/len(losses)
  #print(f"loss after epoch number {epoch} is {mean_loss}")  

In [ ]:
for epoch in range(10):
  print(f'epoch number {epoch}')
  losses=[] #use for calculating loss at each epoch
  #model.train()

 
  checkpoint={'state_dict':model.state_dict(),'optimizer':optimizer.state_dict()}
  save_checkpoint(checkpoint)
  for batch_idx , data in enumerate(train_loader):
    for k, v in data.items():
        data[k] = v.to(device)
    target_tag=data['target_tag']  
    #forward
    _,loss=model(**data)
    #print(scores.size())
    #target_tag=target_tag.view(-1)
    #loss=criterion2(scores,target_tag)

    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

  mean_loss=sum(losses)/len(losses)
  print(f"loss after epoch number {epoch} is {mean_loss}")

epoch number 0
saving checkpoint
loss after epoch number 0 is 0.44657372508911375
epoch number 1
saving checkpoint
loss after epoch number 1 is 0.11070421641917093
epoch number 2
saving checkpoint
loss after epoch number 2 is 0.05248481154124787
epoch number 3
saving checkpoint
loss after epoch number 3 is 0.025369142778283844
epoch number 4
saving checkpoint
loss after epoch number 4 is 0.020110411167389167
epoch number 5
saving checkpoint
loss after epoch number 5 is 0.012990778566384358
epoch number 6
saving checkpoint
loss after epoch number 6 is 0.007952099057502306
epoch number 7
saving checkpoint
loss after epoch number 7 is 0.007720789418420391
epoch number 8
saving checkpoint
loss after epoch number 8 is 0.007263534914550259
epoch number 9
saving checkpoint
loss after epoch number 9 is 0.005139967839477967


In [ ]:
/tags_dict

{'B-LOC ': 5,
 'B-MISC ': 2,
 'B-ORG ': 0,
 'B-PER ': 3,
 'I-LOC ': 8,
 'I-MISC ': 7,
 'I-ORG ': 6,
 'I-PER ': 4,
 'O ': 1}

In [ ]:
model.eval()
tokenizer=BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)
with torch.no_grad():
  list_word=[]
  tokenize_word=[]
  sent='Robert is good boy and he is going to London and working in Google'
  for word in sent.split(' '):
      tokenize_word.extend(tokenizer.tokenize(word))
  tokenize_word=['cls']+tokenize_word+['sep']
  print(f"tokenize words {tokenize_word}")
  for word in sent.split(' '):
    list_word.append(word)
  test_dataset=NerDataset(sents=[list_word],tags=[[0]*len(list_word)],max_len=128)
  data=test_dataset[0] #only single element
  print(data)
  for k , v in data.items():
      data[k]=v.to(device).unsqueeze(0) #to add axis to make input like this for bert [1,128] single sent batch
      #print('final_data',data)
  tag , _  = model(**data)
  #print(tokenize_sent)
  print(tag.size())
  print(tag)
  values=tag.argmax(2).view(-1)
  new_val=values.to('cpu').numpy() #this is use for printing on cpu not on cuda 
  final_values=[]
  for i in range(len(tokenize_word)):
      final_values.append(new_val[i])
  print(f'tokenize sent {tokenize_word}')
  print(f'tags for tokenize word {final_values}')


tokenize words ['cls', 'robe', '##rt', 'is', 'good', 'boy', 'and', 'he', 'is', 'going', 'to', 'lo', '##ndon', 'and', 'working', 'in', 'go', '##og', '##le', 'sep']
{'id': tensor([  101, 11580,  3740,  1110,  1363,  2298,  1105,  1119,  1110,  1280,
         1106, 25338, 17996,  1105,  1684,  1107,  1301,  8032,  1513,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
tags_dict

{'B-LOC ': 5,
 'B-MISC ': 2,
 'B-ORG ': 0,
 'B-PER ': 3,
 'I-LOC ': 8,
 'I-MISC ': 7,
 'I-ORG ': 6,
 'I-PER ': 4,
 'O ': 1}

In [ ]:
def real_time(sent):#this function can be used in real time
    training_model=Model_Normal(num_tags=num_class).to(device)
    checkpoint=torch.load('/content/drive/My Drive/Colab Notebooks/TransformerMachineTranslation/bert_ner_checkpoint/bert_checkpoint_second.pth')
    training_model.load_state_dict(checkpoint['state_dict'])

    tokenizer=BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)
    training_model.eval()
    with torch.no_grad():
      list_word=[]
      tokenize_word=[]
      #sent='Robert is good boy and he is going to London google'
      for word in sent.split(' '):
          tokenize_word.extend(tokenizer.tokenize(word))
      tokenize_word=['cls']+tokenize_word+['sep']
      print(f"tokenize words {tokenize_word}")
      for word in sent.split(' '):
        list_word.append(word)
      test_dataset=NerDataset(sents=[list_word],tags=[[0]*len(list_word)],max_len=128)
      data=test_dataset[0] #only single element
      print(data)
      for k , v in data.items():
          data[k]=v.to(device).unsqueeze(0) #to add axis to make input like this for bert [1,128] single sent batch
          #print('final_data',data)
      tag , _  = training_model(**data)
      #print(tokenize_sent)
      print(tag.size())
      print(tag)
      values=tag.argmax(2).view(-1)
      new_val=values.to('cpu').numpy() #this is use for printing on cpu not on cuda 
      final_values=[]
      for i in range(len(tokenize_word)):
          final_values.append(new_val[i])
      print(f'tokenize sent {tokenize_word}')
      print(f'tags for tokenize word {final_values}')


In [ ]:
real_time('Nitin is going America')

tokenize words ['cls', 'ni', '##tin', 'is', 'going', 'am', '##eric', '##a', 'sep']
{'id': tensor([  101, 11437,  6105,  1110,  1280,  1821, 26237,  1161,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,   

In [ ]:
tags_dict

{'B-LOC ': 5,
 'B-MISC ': 2,
 'B-ORG ': 0,
 'B-PER ': 3,
 'I-LOC ': 8,
 'I-MISC ': 7,
 'I-ORG ': 6,
 'I-PER ': 4,
 'O ': 1}

In [ ]:
#tokenizer=BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)

def real_time(sent,model):
  tokenize_sent=[]
  tokenizer=BertTokenizer.from_pretrained('bert-base-cased',do_lower_case=True)

  device=torch.device('cuda')
  #model=Model(num_tags=9,hidden_size=256,input_size=768)
  checkpoint=torch.load('/content/drive/My Drive/Colab Notebooks/TransformerMachineTranslation/bert_ner_checkpoint/bert_checkpoint_second.pth')
  model.load_state_dict(checkpoint['state_dict'])

  model=model.to(device=device)
  list_word=[]
  words=sent.split(' ')
  for word in words:
    x=tokenizer.tokenize(word)
    tokenize_sent.extend(x)
  tokenize_sent=['CLS'] + tokenize_sent + ['SEP']
  for word in words :
    list_word.append(word)
  test_dataset=NerDataset(sents=[list_word],tags=[[0]*len(list_word)],max_len=128)
  model.eval()
  
  with torch.no_grad():
    data=test_dataset[0] #only single element
    for k , v in data.items():

      data[k]=v.to(device).unsqueeze(0) #to add axis to make input like this for bert [1,128] single sent batch

    print('final_data',data)
    tag = model(**data)
  
    print(tokenize_sent)
    print(tag.argmax(1))
    #print(tag.size())


In [ ]:

#tokenizer.tokenize('My name is nitin I live in India and work in India.Pvt Ltd')

In [ ]:

x=real_time('Nitin sharma is my name',model)

Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      

{'B-LOC ': 5,
 'B-MISC ': 2,
 'B-ORG ': 0,
 'B-PER ': 3,
 'I-LOC ': 8,
 'I-MISC ': 7,
 'I-ORG ': 6,
 'I-PER ': 4,
 'O ': 1}

In [ ]:
x['id'].size()

torch.Size([128])

In [ ]:
x['mask'].size()

torch.Size([128])

In [ ]:
x['token_type_ids'].size()

torch.Size([128])

In [ ]:
import torch

In [ ]:
x=torch.ones(5,20,10)

In [ ]:
x.view(-1)==1

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr